In [6]:
%load_ext autoreload
%autoreload 2


import numpy as np
import pandas as pd
import anndata as ad
import numpy as np
import pandas as pd
import jax.numpy as jnp
import jax
import cellflow
jax.config.update("jax_default_device", jax.devices()[0])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
adata  = ad.read_h5ad('pbmc_cytokines_train.h5ad')

In [8]:
adata

AnnData object with n_obs × n_vars = 9593880 × 2000
    obs: 'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'bc1_wind', 'bc2_wind', 'bc3_wind', 'bc1_well', 'bc2_well', 'bc3_well', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'total_counts_MT', 'pct_counts_MT', 'log1p_total_counts_MT', 'donor', 'cytokine', 'treatment', 'cell_type', 'cytokine_family', 'condition', 'is_control'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'donor_embeddings', 'esm2_embeddings', 'hvg', 'log1p', 'pca'
    obsm: 'X_pca'
    varm: 'PCs', 'X_mean'

In [9]:
cf = cellflow.model.CellFlow(
    adata=adata,
    solver='otfm'
)

In [10]:
cf.prepare_data(
    sample_rep = "X_pca",
    control_key = "is_control",
    perturbation_covariates = {"cytokine_treatment": ("cytokine",)},
    perturbation_covariate_reps = {"cytokine_treatment": "esm2_embeddings"},
    sample_covariates = ["donor"],
    sample_covariate_reps = {"donor": "donor_embeddings"},
    split_covariates = ["donor"],
    max_combination_length = 1,
    null_value = 0.0,
)

[########################################] | 100% Completed | 2.32 sms


/home/icb/selman.ozleyen/miniforge3/envs/cellflow/lib/python3.11/site-packages/jax/_src/numpy/lax_numpy.py:5540: RuntimeWarning: invalid value encountered in cast
  out = np.asarray(object, dtype=dtype)


[########################################] | 100% Completed | 933.38 ms


In [11]:
split_cov_combs = cf.data_manager._get_split_cov_combs(covariate_data=cf.data_manager.adata.obs)
o = cf.data_manager._get_condition_data_old(
    adata=cf.data_manager.adata,
    split_cov_combs=split_cov_combs,
)

/ictstr01/home/icb/selman.ozleyen/projects/cell_flow_perturbation/src/cellflow/data/_datamanager.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _covariate_data["cell_index"] = _covariate_data.index
 86%|████████▌ | 77/90 [00:42<00:05,  2.19it/s]

In [ ]:
n = cf.data_manager._get_condition_data(
    adata=cf.data_manager.adata,
    split_cov_combs=split_cov_combs,
)

[########################################] | 100% Completed | 212.68 ms
[########################################] | 100% Completed | 311.17 ms


In [ ]:
attrs=[
    "split_idx_to_covariates",
    "perturbation_idx_to_covariates",
    "perturbation_idx_to_id",
    "condition_data",  # type: ignore[arg-type]
    "control_to_perturbation",
    "split_covariates_mask",
    "perturbation_covariates_mask",
]
for attr in attrs:
    old_attr = getattr(o, attr)
    new_attr = getattr(n, attr)
    if attr  in {"split_idx_to_covariates"}:
        assert set(old_attr.keys()) == set(new_attr.keys()), f"Mismatch in {attr} keys, len old: {len(old_attr)}, len new: {len(new_attr)}, old: {old_attr.keys()}, new: {new_attr.keys()}"
        for key in old_attr.keys():
            assert new_attr[key] == old_attr[key], f"Mismatch in {attr} for key {key}, old: {old_attr[key]}, new: {new_attr[key]}"

    elif isinstance(old_attr, dict):
        assert set(old_attr.keys()) == set(new_attr.keys()), f"Mismatch in {attr} keys"
        for key in old_attr.keys():
            old_ = np.array(old_attr[key])
            new_ = np.array(new_attr[key])
            assert np.array_equal(old_, new_), f"Mismatch in {attr} for key {key}, old: {old_}, new: {new_}"
    else:
        np_old = np.array(old_attr)
        np_new = np.array(new_attr)
        print(np_old, np_new)
        assert np.array_equal(np_old, np_new), f"Mismatch in {attr} values"

    print(f"Passed {attr} check")

Passed split_idx_to_covariates check
Passed perturbation_idx_to_covariates check
Passed perturbation_idx_to_id check
Passed condition_data check
Passed control_to_perturbation check
[-1 -1 -1 ... -1 -1 -1] [-1 -1 -1 ... -1 -1 -1]
Passed split_covariates_mask check
[766 699 394 ...  79 508 489] [766 699 394 ...  79 508 489]
Passed perturbation_covariates_mask check


In [9]:
o.split_idx_to_covariates

{0: ('Donor1',),
 1: ('Donor2',),
 2: ('Donor3',),
 3: ('Donor4',),
 4: ('Donor5',),
 5: ('Donor6',),
 6: ('Donor7',),
 7: ('Donor8',),
 8: ('Donor9',),
 9: ('Donor10',),
 10: ('Donor11',),
 11: ('Donor12',)}

In [10]:
# Create a mapping between old indices and new indices based on their values
old_to_new_map = {}
for old_idx, cov_value in o.split_idx_to_covariates.items():
    for new_idx, new_cov_value in n.split_idx_to_covariates.items():
        if cov_value == new_cov_value:
            old_to_new_map[old_idx] = new_idx
            break

print("Mapping between indices:", old_to_new_map)

# Transform the old mask to match the new indexing
transformed_mask = np.copy(o.split_covariates_mask)
for i in range(len(transformed_mask)):
    if transformed_mask[i] != -1:  # Only transform valid indices
        transformed_mask[i] = old_to_new_map[transformed_mask[i]]

# Check if they match after transformation
match = np.array_equal(transformed_mask, n.split_covariates_mask)
print(f"Masks match after transformation: {match}")

# Count differences
if not match:
    diff_indices = np.where(transformed_mask != n.split_covariates_mask)[0]
    print(f"Number of differences: {len(diff_indices)} out of {len(transformed_mask)}")
    # Show a few examples
    if len(diff_indices) < 10:
        for i in diff_indices[:5]:
            print(f"Index {i}: Old={transformed_mask[i]}, New={n.split_covariates_mask[i]}")

Mapping between indices: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11}
Masks match after transformation: True


In [11]:
# Analyze what types of differences we're seeing
diff_indices = np.where(transformed_mask != n.split_covariates_mask)[0]

# Check if the differences involve -1 values
neg_one_in_old = np.sum(transformed_mask[diff_indices] == -1)
neg_one_in_new = np.sum(n.split_covariates_mask[diff_indices] == -1)

print(f"Cells with -1 in old mask among differences: {neg_one_in_old}")
print(f"Cells with -1 in new mask among differences: {neg_one_in_new}")

# Distribution of values in the differences
old_values, old_counts = np.unique(transformed_mask[diff_indices], return_counts=True)
new_values, new_counts = np.unique(n.split_covariates_mask[diff_indices], return_counts=True)

print("Distribution in old mask:")
for val, count in zip(old_values, old_counts):
    print(f"  Value {val}: {count} cells")

print("Distribution in new mask:")
for val, count in zip(new_values, new_counts):
    print(f"  Value {val}: {count} cells")

# Sample some specific examples
sample_indices = diff_indices[:5]
print("\nSample differences:")
for idx in sample_indices:
    print(f"Cell {idx}: Old={transformed_mask[idx]}, New={n.split_covariates_mask[idx]}")
    # Optional: If you have donor info, print it
    # donor = cf.data_manager.adata.obs.iloc[idx]["donor"]
    # is_control = cf.data_manager.adata.obs.iloc[idx]["is_control"]
    # print(f"  Donor: {donor}, Is Control: {is_control}")

Cells with -1 in old mask among differences: 0
Cells with -1 in new mask among differences: 0
Distribution in old mask:
Distribution in new mask:

Sample differences:


In [12]:
# Add this debugging code
print("Number of control cells in old implementation:", np.sum(o.split_covariates_mask != -1))
print("Number of control cells in new implementation:", np.sum(n.split_covariates_mask != -1))

# Check if the control identification is the source
adata_controls = cf.data_manager.adata.obs[cf.data_manager._control_key]
print("Total control cells in dataset:", adata_controls.sum())

Number of control cells in old implementation: 31485
Number of control cells in new implementation: 31485
Total control cells in dataset: 31485


In [13]:
(o.split_covariates_mask == -1) & (n.split_covariates_mask != -1)

Array([False, False, False, ..., False, False, False], dtype=bool)

In [14]:
# Add code to see which control cells are being assigned differently
control_cells = cf.data_manager.adata.obs[cf.data_manager._control_key]
control_indices = np.where(control_cells)[0]

# Check differences among control cells only
control_diffs = control_indices[transformed_mask[control_indices] != n.split_covariates_mask[control_indices]]
print(f"Control cells with different assignments: {len(control_diffs)}")

# Check a few examples
for idx in control_diffs[:5]:
    donor = cf.data_manager.adata.obs.iloc[idx]["donor"]
    print(f"Cell {idx}: Donor={donor}, Old={transformed_mask[idx]}, New={n.split_covariates_mask[idx]}")

Control cells with different assignments: 0


In [15]:
len(o.split_covariates_mask), len(n.split_covariates_mask)

(479694, 479694)

In [16]:
# 1. First, check how donor and control values align
print("===== DONOR & CONTROL ALIGNMENT =====")
# Map donor values to split indices in both implementations
donor_to_old_idx = {}
donor_to_new_idx = {}

# Check donor to index mapping in both implementations
for donor in sorted(cf.data_manager.adata.obs["donor"].unique()):
    # Old implementation mapping
    for old_idx, old_cov in o.split_idx_to_covariates.items():
        if old_cov == (donor,):  # Single-value tuple
            donor_to_old_idx[donor] = old_idx
    
    # New implementation mapping
    for new_idx, new_cov in n.split_idx_to_covariates.items():
        if new_cov == (donor,):
            donor_to_new_idx[donor] = new_idx

print("Donor to index mapping - Old:", donor_to_old_idx)
print("Donor to index mapping - New:", donor_to_new_idx)

# 2. Check if the mappings are applied consistently
print("\n===== CONTROL CELL ASSIGNMENTS =====")
control_indices = np.where(cf.data_manager.adata.obs[cf.data_manager._control_key])[0]
sample_indices = control_indices[:10]  # Examine first 10 control cells

print(f"Sample of {len(sample_indices)} control cells:")
for idx in sample_indices:
    donor = cf.data_manager.adata.obs.iloc[idx]["donor"]
    expected_old_idx = donor_to_old_idx.get(donor, "Not found")
    expected_new_idx = donor_to_new_idx.get(donor, "Not found")
    actual_old = o.split_covariates_mask[idx]
    actual_new = n.split_covariates_mask[idx]
    
    match_old = "✓" if actual_old == expected_old_idx else "✗"
    match_new = "✓" if actual_new == expected_new_idx else "✗"
    
    print(f"Cell {idx}: Donor={donor}, Old={actual_old} (Expected: {expected_old_idx}) {match_old}, " +
          f"New={actual_new} (Expected: {expected_new_idx}) {match_new}")

# 3. Examine the specific code paths for control detection
print("\n===== IMPLEMENTATION DIFFERENCES =====")
# Print key stats about the assignments
total_cells = len(cf.data_manager.adata)
print(f"Total cells: {total_cells}")
print(f"Control cells assigned donor in old implementation: {np.sum(o.split_covariates_mask != -1)}")
print(f"Control cells assigned donor in new implementation: {np.sum(n.split_covariates_mask != -1)}")
print(f"Control cells assigned -1 in new implementation: {np.sum((n.split_covariates_mask == -1) & (cf.data_manager.adata.obs[cf.data_manager._control_key]))}")

===== DONOR & CONTROL ALIGNMENT =====
Donor to index mapping - Old: {'Donor1': 0, 'Donor10': 9, 'Donor11': 10, 'Donor12': 11, 'Donor2': 1, 'Donor3': 2, 'Donor4': 3, 'Donor5': 4, 'Donor6': 5, 'Donor7': 6, 'Donor8': 7, 'Donor9': 8}
Donor to index mapping - New: {'Donor1': 0, 'Donor10': 9, 'Donor11': 10, 'Donor12': 11, 'Donor2': 1, 'Donor3': 2, 'Donor4': 3, 'Donor5': 4, 'Donor6': 5, 'Donor7': 6, 'Donor8': 7, 'Donor9': 8}

===== CONTROL CELL ASSIGNMENTS =====
Sample of 10 control cells:
Cell 15: Donor=Donor12, Old=11 (Expected: 11) ✓, New=11 (Expected: 11) ✓
Cell 22: Donor=Donor11, Old=10 (Expected: 10) ✓, New=10 (Expected: 10) ✓
Cell 119: Donor=Donor8, Old=7 (Expected: 7) ✓, New=7 (Expected: 7) ✓
Cell 134: Donor=Donor6, Old=5 (Expected: 5) ✓, New=5 (Expected: 5) ✓
Cell 137: Donor=Donor6, Old=5 (Expected: 5) ✓, New=5 (Expected: 5) ✓
Cell 150: Donor=Donor6, Old=5 (Expected: 5) ✓, New=5 (Expected: 5) ✓
Cell 159: Donor=Donor7, Old=6 (Expected: 6) ✓, New=6 (Expected: 6) ✓
Cell 168: Donor=Donor